In [650]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1660 Ti


## load and normalize data

In [1708]:
class wineDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx]
        return features

In [1709]:
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

train = pd.concat([X_train, y_train], axis=1)
size = train.shape[0]
scaler = StandardScaler()
scaler.fit(train)
train_scale = scaler.transform(train)

train_dataloader = DataLoader(wineDataset(train_scale), batch_size=size, shuffle=True)


In [1710]:
next(enumerate(train_dataloader))[1][0]

tensor([ 0.4204, -0.7225,  0.7442, -0.3951, -0.6249, -0.0828, -0.1893, -1.3851,
        -0.6188, -0.3558,  1.5959,  0.4504], dtype=torch.float64)

## define model

In [1049]:
noise_len = 100

In [1742]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(noise_len, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 12),
        )

    def forward(self, input):
        return self.gen(input)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(12, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.dis(input)
        

## initialize training

In [1743]:
netG = Generator().to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()

real_label = torch.ones((size, 1), dtype=torch.float32, device=device)
fake_label = torch.zeros((size, 1), dtype=torch.float32, device=device)

G_losses = []
D_losses = []

## train

In [1744]:
lr = 0.0003
num_epoch = 50

optimizerD = optim.Adam(netD.parameters(), lr=lr, weight_decay=0)
optimizerG = optim.Adam(netG.parameters(), lr=lr, weight_decay=0)

### train discriminator

In [1815]:
for epoch in range(num_epoch):
    for i, data in enumerate(train_dataloader):
        
        # real data
        netD.zero_grad()
        data = data.to(torch.float32).to(device)
        
        output = netD(data)

        errD_real = criterion(output, real_label)
        errD_real.backward()
        D_x = output.mean().item()

        # fake data
        noise = torch.randn(size, noise_len, device=device)
        fake = netG(noise)
        
        output = netD(fake)

        errD_fake = criterion(output, fake_label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        D_losses.append(errD.item())
        
    if epoch % 5 == 0:
        print('[%d/%d]\tLoss_D: %.4f\tReal mean: %.4f\tFake mean: %.4f' % (epoch, num_epoch, errD.item(), D_x, D_G_z1))





[0/50]	Loss_D: 0.5856	Real mean: 0.7686	Fake mean: 0.2381
[5/50]	Loss_D: 0.5693	Real mean: 0.7713	Fake mean: 0.2301
[10/50]	Loss_D: 0.5618	Real mean: 0.7738	Fake mean: 0.2258
[15/50]	Loss_D: 0.5477	Real mean: 0.7762	Fake mean: 0.2191
[20/50]	Loss_D: 0.5463	Real mean: 0.7787	Fake mean: 0.2212
[25/50]	Loss_D: 0.5499	Real mean: 0.7810	Fake mean: 0.2237
[30/50]	Loss_D: 0.5427	Real mean: 0.7833	Fake mean: 0.2225
[35/50]	Loss_D: 0.5352	Real mean: 0.7853	Fake mean: 0.2188
[40/50]	Loss_D: 0.5277	Real mean: 0.7870	Fake mean: 0.2143
[45/50]	Loss_D: 0.5090	Real mean: 0.7889	Fake mean: 0.2029


### train generator

In [1818]:
for epoch in range(num_epoch):
    for i, data in enumerate(train_dataloader, 0):

        netG.zero_grad()
        
        noise = torch.randn(size, noise_len, device=device)
        fake = netG(noise)
        output = netD(fake)
        
        errG = criterion(output, real_label)
        errG.backward()
        
        D_G_z2 = output.mean().item()
        optimizerG.step()

        G_losses.append(errG.item())

    if epoch % 5 == 0:
        print('[%d/%d]\tLoss_G: %.4f\tFake mean: %.4f' % (epoch, num_epoch, errG.item(), D_G_z2))





[0/50]	Loss_G: 0.2162	Fake mean: 0.8194
[5/50]	Loss_G: 0.1895	Fake mean: 0.8378
[10/50]	Loss_G: 0.1911	Fake mean: 0.8384
[15/50]	Loss_G: 0.1806	Fake mean: 0.8462
[20/50]	Loss_G: 0.1745	Fake mean: 0.8507
[25/50]	Loss_G: 0.1683	Fake mean: 0.8558
[30/50]	Loss_G: 0.1514	Fake mean: 0.8671
[35/50]	Loss_G: 0.1520	Fake mean: 0.8684
[40/50]	Loss_G: 0.1519	Fake mean: 0.8671
[45/50]	Loss_G: 0.1495	Fake mean: 0.8703


## generate 

In [1819]:

with torch.no_grad():
    noise = torch.randn(16, noise_len, device=device)
    fake = netG(noise).cpu()
    fake = scaler.inverse_transform(fake)

fake = pd.DataFrame(fake).round(3)
fake.head(7)


,0,1,2,3,4,5,6,7,8,9,10,11
0,8.617,0.399,0.364,1.524,0.070,16.454,51.202,0.995,3.375,0.641,10.492,6.301
1,11.556,0.571,0.365,3.684,0.100,12.610,9.713,1.001,3.126,0.822,10.983,5.369
2,11.744,0.562,0.377,3.337,0.106,13.745,10.745,1.000,3.150,0.799,11.199,5.568
3,9.058,0.620,0.223,3.162,0.096,4.262,28.233,0.998,3.252,0.589,10.582,5.006
4,10.851,0.575,0.288,3.163,0.110,1.889,15.096,1.000,3.251,0.698,11.250,5.679
5,10.878,0.412,0.489,2.272,0.094,27.109,35.247,0.999,3.200,0.873,10.967,6.570
6,8.691,0.407,0.322,1.721,0.073,19.439,42.546,0.995,3.293,0.640,10.242,6.008


In [539]:
pd.DataFrame(train.values.round(3)).head(20)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.8,0.500,0.17,1.6,0.082,21.0,102.0,0.996,3.39,0.48,9.5,5.0
1,7.6,0.290,0.49,2.7,0.092,25.0,60.0,0.997,3.31,0.61,10.1,6.0
2,6.9,0.400,0.24,2.5,0.083,30.0,45.0,0.996,3.26,0.58,10.0,5.0
3,8.9,0.635,0.37,1.7,0.263,5.0,62.0,0.997,3.00,1.09,9.3,5.0
4,9.4,0.300,0.56,2.8,0.080,6.0,17.0,0.996,3.15,0.92,11.7,8.0
5,8.0,0.710,0.00,2.6,0.080,11.0,34.0,0.998,3.44,0.53,9.5,5.0
6,7.8,0.410,0.68,1.7,0.467,18.0,69.0,0.997,3.08,1.31,9.3,5.0
7,8.4,0.340,0.42,2.1,0.072,23.0,36.0,0.994,3.11,0.78,12.4,6.0
8,6.3,0.760,0.00,2.9,0.072,26.0,52.0,0.994,3.51,0.60,11.5,6.0
9,7.7,0.835,0.00,2.6,0.081,6.0,14.0,0.998,3.30,0.52,9.3,5.0


## save model

In [1820]:
torch.save(netD.state_dict(), './model/netD.pt')
torch.save(netG.state_dict(), './model/netG.pt')

## load model

In [ ]:
netD = Discriminator()
netD.load_state_dict(torch.load('./model/netD.pt'))

netG = Generator()
netG.load_state_dict(torch.load('./model/netG.pt'))

In [35]:
i, data = next(enumerate(train_dataloader))
data

tensor([[10.3000,  0.3200,  0.4500,  ...,  0.8200, 12.6000,  8.0000],
        [ 6.6000,  0.8150,  0.0200,  ...,  0.8900, 12.3000,  7.0000],
        [ 7.5000,  0.7100,  0.0000,  ...,  0.5800, 10.0000,  6.0000],
        ...,
        [ 7.9000,  0.4300,  0.2100,  ...,  0.9100,  9.5000,  5.0000],
        [ 6.7000,  0.4100,  0.4300,  ...,  1.1600, 10.6000,  6.0000],
        [ 7.0000,  0.3800,  0.4900,  ...,  0.7700, 11.4000,  6.0000]],
       dtype=torch.float64)

In [28]:
data.dtype

torch.float64

In [36]:
netD(data.to(device).float()).shape

torch.Size([512, 1])

In [37]:
real_label.shape

torch.Size([512, 1])

In [40]:
noise = torch.randn(512, 64, device=device)
fake = netG(noise)
fake.shape

torch.Size([512, 12])